## Data Description

You are provided with 25,010 poker hands in train.csv and 1,000,000 in test.csv. Each hand consists of five cards with a given suit and rank, drawn from a standard deck of 52. Suits and ranks are represented as ordinal categories:

```
S1 “Suit of card #1”
Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}
C1 “Rank of card #1”
Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

...

S5 “Suit of card #5”
C5 “Rank of card #5”
```

Each row in the training set has the accompanying class label for the poker hand it comprises. The hands are omitted from the test set and must be predicted by participants. Hands are classified into the following ordinal categories:


```
0: Nothing in hand; not a recognized poker hand 
1: One pair; one pair of equal ranks within five cards
2: Two pairs; two pairs of equal ranks within five cards
3: Three of a kind; three equal ranks within five cards
4: Straight; five cards, sequentially ranked with no gaps
5: Flush; five cards with the same suit
6: Full house; pair + different rank three of a kind
7: Four of a kind; four equal ranks within five cards
8: Straight flush; straight + flush
9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush
```
Note that the Straight flush and Royal flush hands are not representative of
the true domain because they have been over-sampled. The straight flush
is 14.43 times more likely to occur in the training set, while the royal flush is 129.82 times more likely.

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from functools import reduce

## Train data

In [2]:
suits = ['S1','S2','S3','S4','S5']
cards = ['C1','C2','C3','C4','C5']

In [3]:
train_dataset = pd.read_csv('train.csv', delimiter=',')

In [4]:
train_dataset.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,4,9,2,1,2,2,4,7,2,8,0
1,1,4,3,6,1,12,3,11,2,7,0
2,1,11,4,1,3,7,4,11,2,1,2
3,2,9,2,4,3,6,1,9,4,9,3
4,1,8,2,4,2,11,2,2,2,1,0


In [5]:
def two_class_dataset(data, column, non_zero_class):
    data = data.copy()
    data[column] = (data[column] == non_zero_class) * 1
    return data

In [6]:
straights_train_dataset = two_class_dataset(train_dataset, 'hand', 4)

In [7]:
straights_train_dataset.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,hand
0,4,9,2,1,2,2,4,7,2,8,0
1,1,4,3,6,1,12,3,11,2,7,0
2,1,11,4,1,3,7,4,11,2,1,0
3,2,9,2,4,3,6,1,9,4,9,0
4,1,8,2,4,2,11,2,2,2,1,0


In [8]:
data = straights_train_dataset.drop('hand', axis=1)
train_labels = straights_train_dataset['hand']

In [9]:
def data_hotencode(data, columns, encoder, mode='transform'):
    encoded = None
    if (mode == 'fit'):
        encoded = encoder.fit_transform(data[columns])
    elif (mode == 'transform'):
        encoded = encoder.transform(data[columns])
    return pd.concat([data.drop(columns, axis=1), pd.DataFrame(encoded.toarray())], axis=1)

In [10]:
encoder = OneHotEncoder()
data_encoded = data_hotencode(data, suits, encoder, 'fit')
print(data_encoded.shape)

data_encoded.head()

(25010, 25)


,C1,C2,C3,C4,C5,0,1,2,3,4,...,10,11,12,13,14,15,16,17,18,19
0,9,1,2,7,8,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,4,6,12,11,7,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,11,1,7,11,1,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,9,4,6,9,9,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8,4,11,2,1,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
def normalize(data, columns, mean, std):
    data_normalized = (data[columns] - mean) / std
    to_return = pd.concat([data_normalized, data.drop(columns, axis=1)], axis=1)
    to_return.columns = list(range(to_return.shape[1]))
    return to_return

In [12]:
mean = data_encoded[cards].mean().mean()
std = pd.Series(data_encoded[cards].values.flatten()).std()
data_normalized = normalize(data_encoded, cards, mean, std)
data_normalized.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.537109,-1.596136,-1.329480,0.003798,0.270453,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-0.796169,-0.262858,1.337076,1.070420,0.003798,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.070420,-1.596136,0.003798,1.070420,-1.596136,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,0.537109,-0.796169,-0.262858,0.537109,0.537109,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.270453,-0.796169,1.070420,-1.329480,-1.596136,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [13]:
data_normalized.shape

(25010, 25)

#### Test function

In [14]:
def calc_hand_label(hand):    
    def f(hand):
        suits_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,0], {})
        ranks_hist = reduce(lambda d,x: {**d, **{x: (d.get(x, 0) + 1)}}, hand[:,1], {})
        
        if len(ranks_hist.values()) < 5:
            if len(ranks_hist.values()) == 2:
                if max(list(ranks_hist.values())) == 4:
                    return 7
    #                 print('7: Four of a kind; four equal ranks within five cards')
                else:
                    return 6
    #                 print('6 Full house; pair + different rank three of a kind')
            elif len(ranks_hist.values()) == 3:
                if max(list(ranks_hist.values())) == 3:
                    return 3
    #                 print('3: Three of a kind; three equal ranks within five cards')
                else:
                    return 2
    #                 print('2: Two pairs; two pairs of equal ranks within five cards')
            else:
                return 1
    #             print('1: One pair; one pair of equal ranks within five cards')
        else:
            if len(suits_hist.values()) == 1:
                if max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                    if max(list(ranks_hist.keys())) == 13:
                        return 9
    #                     print('9: Royal flush; {Ace, King, Queen, Jack, Ten} + flush')
                    else:
                        return 8
    #                     print('8: Straight flush; straight + flush')
                else:
                    return 5
    #                 print('5: Flush; five cards with the same suit')
            elif max(list(ranks_hist.keys())) -  min(list(ranks_hist.keys())) == 4:
                return 4    
    #             print('4: Straight; five cards, sequentially ranked with no gaps')
            else:
                return 0
                print('0: Nothing in hand; not a recognized poker hand ')
                
    hand = np.array(hand).reshape(5,2)
    res1 = f(hand)
    
    hand[:,1] = list(map(lambda x: 13 if (x == 1) else x-1, hand[:,1]))
    res2 = f(hand)
    
    if res1 == 9 and res2 == 8:
        return res2
    else:
        return max(res1, res2)

#### Encode cards

In [15]:
def card_encode(s, c):
    cards=["A","2","3","4","5","6","7","8","9","10","J","Q","K"]
    suits="♥♦♣♠"
    return cards[c - 1] + suits[s - 1]

def hand_encode(hand):
    return ', '.join(map(lambda x: card_encode(hand[x*2], hand[x*2+1]),range(5)))

print(hand_encode([4,9,2,1,2,2,4,7,2,8]))
print(hand_encode([2,9,2,4,3,6,1,9,4,9]))

9♠, A♦, 2♦, 7♠, 8♦
9♦, 4♦, 6♣, 9♥, 9♠


## Neural Network

In [21]:
from time import time
from keras.callbacks import TensorBoard

model = keras.Sequential()

#input
model.add(keras.layers.Dense(100, activation='relu', input_shape=(25,)))
#model.add(keras.layers.Dropout(0.1))

model.add(keras.layers.Dense(200, activation='relu'))
#model.add(keras.layers.Dropout(0.05))
model.add(keras.layers.Dense(50, activation='relu'))
model.add(keras.layers.Dense(25, activation='relu'))

# Add a softmax layer with 10 output units:
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=tf.train.AdamOptimizer(0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(np.array(data_normalized), np.array(train_labels), epochs=100, batch_size=512)

Using TensorFlow backend.


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 19us/step - loss: 0.2391 - acc: 0.9495

Epoch 2/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 12us/step - loss: 0.0242 - acc: 0.9963

Epoch 3/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/step - loss: 0.0235 - acc: 0.9963

Epoch 4/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 0.0231 - acc: 0.9963

Epoch 5/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 0.0227 - acc: 0.9963

Epoch 6/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 0.0223 - acc: 0.9963

Epoch 7/100
25010/25010 [==========================

25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 8.8730e-04 - acc: 0.9998

Epoch 55/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 9.8860e-04 - acc: 0.9998

Epoch 56/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 8us/step - loss: 8.7349e-04 - acc: 0.9996

Epoch 57/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 0.0012 - acc: 0.9995

Epoch 58/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 11us/step - loss: 7.4437e-04 - acc: 0.9998

Epoch 59/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 9us/step - loss: 2.8892e-04 - acc: 1.0000

Epoch 60/100
25010/25010 [==============================]25010/25010 [==============================] - 0s 10us/ste

## Test data

In [16]:
test_dataset = pd.read_csv('test.csv', delimiter=',')
test_data = test_dataset.drop('id', axis=1)

In [17]:
test_labels_full = test_data.apply(calc_hand_label, axis=1)

In [18]:
test_labels_4 = (test_labels_full == 4) * 1
test_labels_4.sum()

3885

In [19]:
test_data_encoded = normalize(
    data_hotencode(test_data, suits, encoder),
    cards,
    mean,
    std,
)

## Predictions

In [23]:
pretictions_prob = model.predict(np.array(test_data_encoded))

In [29]:
pretictions = ((pretictions_prob > 0.5) * 1).reshape(pretictions.shape[0])

In [30]:
pretictions

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
predictions_data = pd.concat([
    test_dataset,
    pd.DataFrame({'pretictions_prob': pretictions_prob.reshape(pretictions_prob.shape[0])}),
    pd.DataFrame({'predictions': pretictions}),
    pd.DataFrame({'labels': test_labels_4}),
], axis=1)

In [44]:
wrong_answers = predictions_data[predictions_data['predictions'] != predictions_data['labels']]
wrong_answers.shape

(4566, 14)

In [45]:
wrong_answers['labels'].value_counts()

1    3849
0     717
Name: labels, dtype: int64

In [46]:
wrong_answers

,id,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,pretictions_prob,predictions,labels
34,35,3,13,3,12,2,10,2,1,1,11,8.705573e-15,0,1
917,918,1,3,4,6,4,7,2,5,2,4,2.260301e-08,0,1
1146,1147,4,4,1,3,3,6,1,2,2,5,6.285114e-08,0,1
1550,1551,1,8,1,6,2,5,3,9,3,7,1.237397e-06,0,1
1610,1611,1,6,3,2,4,5,2,4,1,3,2.951764e-06,0,1
1766,1767,4,11,1,9,2,8,3,7,4,10,4.243045e-09,0,1
1818,1819,3,1,1,11,3,10,4,12,2,13,1.179397e-10,0,1
1912,1913,3,6,2,4,1,3,1,5,2,2,1.640954e-03,0,1
2156,2157,4,11,1,8,1,9,1,10,1,12,1.001730e-10,0,1
2398,2399,1,6,2,3,2,5,4,6,1,4,9.123169e-01,1,0
